In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel("EVO.xlsx")

df = df[:20]
df = df.drop(columns=['Reviewers'])


M = df.shape[1]  # Number of papers
N = df.shape[0]  # Number of reviewers

simmatrix = np.zeros((M, N), dtype=float)


PREFERENCE_SCORES = {
    "Conflict of Interest": -100,
    "None": -100,
    None: -100,
    "Moderate": 1,
    "Moderate*": 1,
    "Moderate**": 1,
    "Considerable": 5,
    "Considerable*": 5,
    "Considerable**": 5
}

for paper_idx in range(M):
    for rev_idx in range(N):
        cell_value = df.iat[rev_idx, paper_idx]
        if pd.isna(cell_value) or str(cell_value).strip() == "":
            # If the cell is empty, treat as "None"
            score_str = "None"
        else:
            score_str = str(cell_value).strip()

        # Lookup the score, default to -1 if not in map
        score = PREFERENCE_SCORES.get(score_str, -100)
        simmatrix[paper_idx, rev_idx] = score

np.set_printoptions(threshold=np.inf)
simmatrix = np.transpose(simmatrix)##always transpose the matrix



from gurobipy import Model, GRB
import numpy as np
import time
from itertools import product

class auto_assigner:
    _EPS = 1e-3
    
    def __init__(self, simmatrix, demand=2, ability=100, function=lambda x: x,
                 iter_limit=np.inf, time_limit=np.inf):
        self.simmatrix = simmatrix
        self.numrev = simmatrix.shape[0]
        self.numpapers = simmatrix.shape[1]
        
        # If ability is just an integer, replicate for all reviewers
        if isinstance(ability, int):
            self.ability = ability * np.ones(self.numrev, dtype=int)
        else:
            self.ability = np.array(ability, dtype=int)
        
        self.demand = demand
        self.function = function
        if iter_limit < 1:
            raise ValueError('Maximum number of iterations must be at least 1')
        self.iter_limit = iter_limit
        self.time_limit = time_limit
        
    def _initialize_model(self):
        self._problem = Model()
        self._problem.setParam('OutputFlag', False)

        # Source to reviewers capacity
        self._source_vars = self._problem.addVars(
            self.numrev, vtype=GRB.CONTINUOUS, 
            lb=0.0, ub=self.ability, name='reviewers'
        )

        # Papers to sink capacity
        self._sink_vars = self._problem.addVars(
            self.numpapers, vtype=GRB.CONTINUOUS,
            lb=0.0, ub=self.demand, name='papers'
        )

        # *** Use BINARY assignment variables so each (reviewer,paper) is either 0 or 1
        self._mix_vars = self._problem.addVars(
            self.numrev, self.numpapers, 
            vtype=GRB.BINARY, 
            lb=0.0, ub=1.0,
            name='assignment'
        )
        self._problem.update()

        # Flow balance constraints
        self._balance_reviewers = self._problem.addConstrs(
            (self._source_vars[i] == self._mix_vars.sum(i, '*') 
             for i in range(self.numrev))
        )
        self._balance_papers = self._problem.addConstrs(
            (self._sink_vars[i] == self._mix_vars.sum('*', i)
             for i in range(self.numpapers))
        )
        self._problem.update()

    def _ranking_of_pairs(self, simmatrix):
        # Only consider pairs where score >= 0
        pairs = [(r, p) for r in range(self.numrev)
                 for p in range(self.numpapers)
                 if simmatrix[r, p] >= 0]
        return sorted(pairs, key=lambda x: simmatrix[x[0], x[1]], reverse=True)

    def _subroutine(self, simmatrix, kappa, abilities, not_assigned, lower_bound, *args):
 

        # 1. First objective: maximize total flow (just to ensure we can get kappa * #papers).
        self._problem.setObjective(
            sum(self._source_vars[i] for i in range(self.numrev)),
            GRB.MAXIMIZE
        )

        # 2. Papers not yet assigned get an upper bound = kappa (we want exactly kappa, but LB=0 for now)
        for paper in not_assigned:
            self._sink_vars[paper].ub = kappa
            self._sink_vars[paper].lb = 0

        # 3. Update each reviewer’s capacity
        for reviewer in range(self.numrev):
            self._source_vars[reviewer].ub = abilities[reviewer]

        # 4. Get the sorted (reviewer, paper) pairs by descending sim score (only ≥ 0)
        sorted_pairs = self._ranking_of_pairs(simmatrix)

        # If an upper bound was passed in via *args, use it; otherwise default
        if args != ():
            upper_bound = args[0]
        else:
            upper_bound = len(sorted_pairs)

        current_solution = 0
        maxflow = -1
        one_iteration_done = False

        while lower_bound < upper_bound or not one_iteration_done:
            one_iteration_done = True
            prev_solution = current_solution
            current_solution = lower_bound + (upper_bound - lower_bound) // 2

            if current_solution == prev_solution:
                if maxflow < len(not_assigned) * kappa and current_solution == lower_bound:
                    current_solution += 1
                    lower_bound += 1
                else:
                    raise ValueError('An error occured1')

            if current_solution > prev_solution:
                for cur_pair in sorted_pairs[prev_solution : current_solution]:
                    self._mix_vars[cur_pair[0], cur_pair[1]].ub = 1
            else:
                for cur_pair in sorted_pairs[current_solution : prev_solution]:
                    self._mix_vars[cur_pair[0], cur_pair[1]].ub = 0

            self._problem.optimize()
            maxflow = self._problem.objVal

            if maxflow == len(not_assigned) * kappa:
                upper_bound = current_solution
            elif maxflow < len(not_assigned) * kappa:
                lower_bound = current_solution
            else:
                raise ValueError('An error occured2')

        if maxflow != len(not_assigned) * kappa or lower_bound != current_solution:
            print(f"Debug Info: maxflow={maxflow}, required_flow={len(not_assigned) * kappa}, lower_bound={lower_bound}, current_solution={current_solution}")
            raise ValueError('An error occured3')

        for paper in not_assigned:
            self._sink_vars[paper].lb = kappa

        self._problem.setObjective(sum([sum([simmatrix[reviewer, paper] * self._mix_vars[reviewer, paper]
                                            for paper in not_assigned])
                                        for reviewer in range(self.numrev)]), GRB.MAXIMIZE)
        self._problem.optimize()

        assignment = {}
        for paper in not_assigned:
            assignment[paper] = []
        for reviewer in range(self.numrev):
            for paper in not_assigned:
                if self._mix_vars[reviewer, paper].X == 1:
                    assignment[paper] += [reviewer]
                if np.abs(self._mix_vars[reviewer, paper].X - int(self._mix_vars[reviewer, paper].X)) > self._EPS:
                    raise ValueError('Error with rounding -- please check that demand and ability are integal')
                self._mix_vars[reviewer, paper].ub = 0
        self._problem.update()

        return assignment, current_solution


    # Join two assignments
    @staticmethod
    def _join_assignment(assignment1, assignment2):
        assignment = {}
        for paper in assignment1:
            assignment[paper] = assignment1[paper] + assignment2[paper]
        return assignment

    # Compute fairfness
    def quality(self, assignment, *args):
        qual = np.inf
        if args != ():
            paper = args[0]
            return np.sum([self.function(self.simmatrix[reviewer, paper]) for reviewer in assignment[paper]])
        else:
            for paper in assignment:
                if qual > sum([self.function(self.simmatrix[reviewer, paper]) for reviewer in assignment[paper]]):
                    qual = np.sum([self.function(self.simmatrix[reviewer, paper]) for reviewer in assignment[paper]])
        return qual
##main algo
    def _fair_assignment(self):
        
        # Counter for number of performed iterations
        iter_counter = 0
        # Start time
        start_time = time.time()
        
        current_best = None
        current_best_score = 0
        local_simmatrix = self.simmatrix.copy()
        local_abilities = self.ability
        not_assigned = set(range(self.numpapers))
        final_assignment = {}

        # one iteration of Steps 2 to 7 of the algorithm
        while not_assigned != set() and iter_counter < self.iter_limit and (time.time() < start_time + self.time_limit or iter_counter == 0):
            
            iter_counter += 1
            
            lower_bound = 0
            upper_bound = len(not_assigned) * self.numrev

            # Step 2
            for kappa in range(1, self.demand + 1):

                # Step 2(a)
                tmp_abilities = local_abilities.copy()
                tmp_simmatrix = local_simmatrix.copy()

                # Step 2(b)
                assignment1, lower_bound = self._subroutine(tmp_simmatrix, kappa, tmp_abilities, not_assigned,
                                                            lower_bound, upper_bound)

                # Step 2(c)
                for paper in assignment1:
                    for reviewer in assignment1[paper]:
                        tmp_simmatrix[reviewer, paper] = -1
                        tmp_abilities[reviewer] -= 1

                # Step 2(d)
                assignment2 = self._subroutine(tmp_simmatrix, self.demand - kappa, tmp_abilities, not_assigned,
                                               lower_bound, upper_bound)[0]

                # Step 2(e)
                assignment = self._join_assignment(assignment1, assignment2)

                # Keep track of the best candidate assignment (including the one from the prev. iteration)
                if self.quality(assignment) > current_best_score or current_best_score == 0:
                    current_best = assignment
                    current_best_score = self.quality(assignment)

            # Steps 4 to 6
            for paper in not_assigned.copy():
                # For every paper not yet fixed in the final assignment we update the assignment
                final_assignment[paper] = current_best[paper]
                # Find the most worst-off paper
                if self.quality(current_best, paper) == current_best_score:
                    # Delete it from current candidate assignment and from the set of papers which are
                    # not yet fixed in the final output
                    del current_best[paper]
                    not_assigned.discard(paper)
                    # This paper is now fixed in the final assignment

                    # Update abilities of reviewers
                    for reviewer in range(self.numrev):
                        # edges adjunct to the vertex of the most worst-off papers
                        # will not be used in the flow network any more
                        local_simmatrix[reviewer, paper] = -1
                        self._mix_vars[reviewer, paper].ub = 0
                        self._mix_vars[reviewer, paper].lb = 0
                        if reviewer in final_assignment[paper]:
                            local_abilities[reviewer] -= 1
            
            current_best_score = self.quality(current_best)
            self._problem.update()

        self.fa = final_assignment
        self.best_quality = self.quality(final_assignment)

    def fair_assignment(self):
        self._initialize_model()
        self._fair_assignment()  # same logic as your original
        # final result is in self.fa
        print(self.fa)
        print(self.best_quality)
        return self.fa
        
res = auto_assigner(simmatrix=simmatrix,
    demand=2,           # e.g., each paper needs exactly 2 reviewers
    ability=8,          # e.g., each reviewer can review up to 3 papers
    function=lambda x: x,   # identity: we want to sum raw scores
    iter_limit=900000,      # or whatever you need
    time_limit=9000      # 5-minute time limit, for instance
).fair_assignment()


maxkeys = max(res.items())
maxkeys
ndf = pd.DataFrame(columns=df.columns)
nameevodf = pd.DataFrame(columns=df.columns)
for col_idx, (row1, row2) in res.items():
    if col_idx < len(df.columns):
        col_name = df.columns[col_idx]
        ndf.at[0, col_name] = df.iloc[row1, col_idx]
        ndf.at[1, col_name] = df.iloc[row2, col_idx]
   


withnameoriginaldf = pd.read_excel("EVO.xlsx")
col_spc = 1
for col_spc, (row1, row2) in res.items():
    if col_spc < len(df.columns):
        col_name = df.columns[col_spc]
        nameevodf.at[0, col_name] = withnameoriginaldf.iloc[row1, 0]
        nameevodf.at[1, col_name] = withnameoriginaldf.iloc[row2, 0]

nameevodf.to_excel('final_result.xlsx')


Restricted license - for non-production use only - expires 2026-11-23
{0: [0, 11], 1: [4, 16], 2: [3, 13], 3: [1, 19], 4: [7, 16], 5: [3, 6], 6: [8, 12], 7: [1, 14], 8: [7, 19], 9: [1, 19], 10: [5, 16], 11: [0, 18], 12: [13, 16], 13: [1, 3], 14: [6, 12], 15: [3, 12], 16: [1, 18], 17: [6, 12], 18: [1, 5], 19: [0, 14], 20: [16, 17], 21: [5, 8], 22: [0, 7], 23: [5, 16], 24: [1, 8], 25: [0, 4], 26: [2, 12], 27: [7, 11], 28: [11, 15], 29: [5, 15], 30: [13, 14], 31: [2, 18], 32: [2, 18], 33: [15, 16], 34: [5, 11], 35: [13, 17], 36: [2, 12], 37: [7, 15], 38: [5, 11], 39: [5, 14], 40: [3, 17], 41: [9, 10], 42: [12, 18], 43: [4, 15], 44: [13, 17], 45: [2, 18], 46: [0, 7], 47: [7, 18], 48: [17, 18], 49: [6, 14], 50: [0, 11], 51: [2, 3], 52: [3, 6], 53: [2, 6], 54: [11, 14], 55: [6, 11], 56: [0, 16], 57: [1, 7], 58: [2, 3], 59: [6, 12]}
2.0


In [24]:
simmatrix


array([[   5., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100., -100.,    1., -100., -100., -100., -100., -100., -100.,
        -100.,    1., -100., -100.,    1., -100., -100.,    5., -100.,
        -100., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100.,    1., -100., -100., -100.,    1., -100., -100., -100.,
        -100., -100.,    1., -100., -100., -100.],
       [   1., -100.,    1.,    5.,    1., -100., -100.,    5.,    1.,
           5., -100.,    1.,    1.,    5., -100., -100.,    5., -100.,
           1.,    1.,    1.,    1., -100., -100.,    5., -100., -100.,
           1.,    1., -100., -100., -100., -100., -100., -100.,    1.,
        -100.,    1., -100.,    1., -100., -100.,    1., -100., -100.,
        -100.,    1., -100., -100., -100., -100., -100., -100., -100.,
        -100.,    1.,    1.,    1., -100., -100.],
       [-100., -100., -100., -100., -100., -10

In [14]:
from gurobipy import Model, GRB
import numpy as np
import time
from itertools import product

class auto_assigner:
    _EPS = 1e-3
    
    def __init__(self, simmatrix, demand=2, ability=100, function=lambda x: x,
                 iter_limit=np.inf, time_limit=np.inf):
        self.simmatrix = simmatrix
        self.numrev = simmatrix.shape[0]
        self.numpapers = simmatrix.shape[1]
        
        # If ability is just an integer, replicate for all reviewers
        if isinstance(ability, int):
            self.ability = ability * np.ones(self.numrev, dtype=int)
        else:
            self.ability = np.array(ability, dtype=int)
        
        self.demand = demand
        self.function = function
        if iter_limit < 1:
            raise ValueError('Maximum number of iterations must be at least 1')
        self.iter_limit = iter_limit
        self.time_limit = time_limit
        
    def _initialize_model(self):
        self._problem = Model()
        self._problem.setParam('OutputFlag', False)

        # Source to reviewers capacity
        self._source_vars = self._problem.addVars(
            self.numrev, vtype=GRB.CONTINUOUS, 
            lb=0.0, ub=self.ability, name='reviewers'
        )

        # Papers to sink capacity
        self._sink_vars = self._problem.addVars(
            self.numpapers, vtype=GRB.CONTINUOUS,
            lb=0.0, ub=self.demand, name='papers'
        )

        # *** Use BINARY assignment variables so each (reviewer,paper) is either 0 or 1
        self._mix_vars = self._problem.addVars(
            self.numrev, self.numpapers, 
            vtype=GRB.BINARY, 
            lb=0.0, ub=1.0,
            name='assignment'
        )
        self._problem.update()

        # Flow balance constraints
        self._balance_reviewers = self._problem.addConstrs(
            (self._source_vars[i] == self._mix_vars.sum(i, '*') 
             for i in range(self.numrev))
        )
        self._balance_papers = self._problem.addConstrs(
            (self._sink_vars[i] == self._mix_vars.sum('*', i)
             for i in range(self.numpapers))
        )
        self._problem.update()

    def _ranking_of_pairs(self, simmatrix):
        # Only consider pairs where score >= 0
        pairs = [(r, p) for r in range(self.numrev)
                 for p in range(self.numpapers)
                 if simmatrix[r, p] >= 0]
        return sorted(pairs, key=lambda x: simmatrix[x[0], x[1]], reverse=True)

    def _subroutine(self, simmatrix, kappa, abilities, not_assigned, lower_bound, *args):
 

        # 1. First objective: maximize total flow (just to ensure we can get kappa * #papers).
        self._problem.setObjective(
            sum(self._source_vars[i] for i in range(self.numrev)),
            GRB.MAXIMIZE
        )

        # 2. Papers not yet assigned get an upper bound = kappa (we want exactly kappa, but LB=0 for now)
        for paper in not_assigned:
            self._sink_vars[paper].ub = kappa
            self._sink_vars[paper].lb = 0

        # 3. Update each reviewer’s capacity
        for reviewer in range(self.numrev):
            self._source_vars[reviewer].ub = abilities[reviewer]

        # 4. Get the sorted (reviewer, paper) pairs by descending sim score (only ≥ 0)
        sorted_pairs = self._ranking_of_pairs(simmatrix)

        # If an upper bound was passed in via *args, use it; otherwise default
        if args != ():
            upper_bound = args[0]
        else:
            upper_bound = len(sorted_pairs)

        current_solution = 0
        maxflow = -1
        one_iteration_done = False

        while lower_bound < upper_bound or not one_iteration_done:
            one_iteration_done = True
            prev_solution = current_solution
            current_solution = lower_bound + (upper_bound - lower_bound) // 2

            if current_solution == prev_solution:
                if maxflow < len(not_assigned) * kappa and current_solution == lower_bound:
                    current_solution += 1
                    lower_bound += 1
                else:
                    raise ValueError('An error occured1')

            if current_solution > prev_solution:
                for cur_pair in sorted_pairs[prev_solution : current_solution]:
                    self._mix_vars[cur_pair[0], cur_pair[1]].ub = 1
            else:
                for cur_pair in sorted_pairs[current_solution : prev_solution]:
                    self._mix_vars[cur_pair[0], cur_pair[1]].ub = 0

            self._problem.optimize()
            maxflow = self._problem.objVal

            if maxflow == len(not_assigned) * kappa:
                upper_bound = current_solution
            elif maxflow < len(not_assigned) * kappa:
                lower_bound = current_solution
            else:
                raise ValueError('An error occured2')

        if maxflow != len(not_assigned) * kappa or lower_bound != current_solution:
            print(f"Debug Info: maxflow={maxflow}, required_flow={len(not_assigned) * kappa}, lower_bound={lower_bound}, current_solution={current_solution}")
            raise ValueError('An error occured3')

        for paper in not_assigned:
            self._sink_vars[paper].lb = kappa

        self._problem.setObjective(sum([sum([simmatrix[reviewer, paper] * self._mix_vars[reviewer, paper]
                                            for paper in not_assigned])
                                        for reviewer in range(self.numrev)]), GRB.MAXIMIZE)
        self._problem.optimize()

        assignment = {}
        for paper in not_assigned:
            assignment[paper] = []
        for reviewer in range(self.numrev):
            for paper in not_assigned:
                if self._mix_vars[reviewer, paper].X == 1:
                    assignment[paper] += [reviewer]
                if np.abs(self._mix_vars[reviewer, paper].X - int(self._mix_vars[reviewer, paper].X)) > self._EPS:
                    raise ValueError('Error with rounding -- please check that demand and ability are integal')
                self._mix_vars[reviewer, paper].ub = 0
        self._problem.update()

        return assignment, current_solution


    # Join two assignments
    @staticmethod
    def _join_assignment(assignment1, assignment2):
        assignment = {}
        for paper in assignment1:
            assignment[paper] = assignment1[paper] + assignment2[paper]
        return assignment

    # Compute fairfness
    def quality(self, assignment, *args):
        qual = np.inf
        if args != ():
            paper = args[0]
            return np.sum([self.function(self.simmatrix[reviewer, paper]) for reviewer in assignment[paper]])
        else:
            for paper in assignment:
                if qual > sum([self.function(self.simmatrix[reviewer, paper]) for reviewer in assignment[paper]]):
                    qual = np.sum([self.function(self.simmatrix[reviewer, paper]) for reviewer in assignment[paper]])
        return qual
##main algo
    def _fair_assignment(self):
        
        # Counter for number of performed iterations
        iter_counter = 0
        # Start time
        start_time = time.time()
        
        current_best = None
        current_best_score = 0
        local_simmatrix = self.simmatrix.copy()
        local_abilities = self.ability
        not_assigned = set(range(self.numpapers))
        final_assignment = {}

        # one iteration of Steps 2 to 7 of the algorithm
        while not_assigned != set() and iter_counter < self.iter_limit and (time.time() < start_time + self.time_limit or iter_counter == 0):
            
            iter_counter += 1
            
            lower_bound = 0
            upper_bound = len(not_assigned) * self.numrev

            # Step 2
            for kappa in range(1, self.demand + 1):

                # Step 2(a)
                tmp_abilities = local_abilities.copy()
                tmp_simmatrix = local_simmatrix.copy()

                # Step 2(b)
                assignment1, lower_bound = self._subroutine(tmp_simmatrix, kappa, tmp_abilities, not_assigned,
                                                            lower_bound, upper_bound)

                # Step 2(c)
                for paper in assignment1:
                    for reviewer in assignment1[paper]:
                        tmp_simmatrix[reviewer, paper] = -1
                        tmp_abilities[reviewer] -= 1

                # Step 2(d)
                assignment2 = self._subroutine(tmp_simmatrix, self.demand - kappa, tmp_abilities, not_assigned,
                                               lower_bound, upper_bound)[0]

                # Step 2(e)
                assignment = self._join_assignment(assignment1, assignment2)

                # Keep track of the best candidate assignment (including the one from the prev. iteration)
                if self.quality(assignment) > current_best_score or current_best_score == 0:
                    current_best = assignment
                    current_best_score = self.quality(assignment)

            # Steps 4 to 6
            for paper in not_assigned.copy():
                # For every paper not yet fixed in the final assignment we update the assignment
                final_assignment[paper] = current_best[paper]
                # Find the most worst-off paper
                if self.quality(current_best, paper) == current_best_score:
                    # Delete it from current candidate assignment and from the set of papers which are
                    # not yet fixed in the final output
                    del current_best[paper]
                    not_assigned.discard(paper)
                    # This paper is now fixed in the final assignment

                    # Update abilities of reviewers
                    for reviewer in range(self.numrev):
                        # edges adjunct to the vertex of the most worst-off papers
                        # will not be used in the flow network any more
                        local_simmatrix[reviewer, paper] = -1
                        self._mix_vars[reviewer, paper].ub = 0
                        self._mix_vars[reviewer, paper].lb = 0
                        if reviewer in final_assignment[paper]:
                            local_abilities[reviewer] -= 1
            
            current_best_score = self.quality(current_best)
            self._problem.update()

        self.fa = final_assignment
        self.best_quality = self.quality(final_assignment)

    def fair_assignment(self):
        self._initialize_model()
        self._fair_assignment()  # same logic as your original
        # final result is in self.fa
        print(self.fa)
        print(self.best_quality)
        return self.fa
        
res = auto_assigner(simmatrix=simmatrix,
    demand=2,           # e.g., each paper needs exactly 2 reviewers
    ability=8,          # e.g., each reviewer can review up to 3 papers
    function=lambda x: x,   # identity: we want to sum raw scores
    iter_limit=900000,      # or whatever you need
    time_limit=9000      # 5-minute time limit, for instance
).fair_assignment()


maxkeys = max(res.items())
maxkeys
ndf = pd.DataFrame(columns=df.columns)
nameevodf = pd.DataFrame(columns=df.columns)
for col_idx, (row1, row2) in res.items():
    if col_idx < len(df.columns):
        col_name = df.columns[col_idx]
        ndf.at[0, col_name] = df.iloc[row1, col_idx]
        ndf.at[1, col_name] = df.iloc[row2, col_idx]
   


withnameoriginaldf = pd.read_excel("EVO.xlsx")
col_spc = 1
for col_spc, (row1, row2) in res.items():
    if col_spc < len(df.columns):
        col_name = df.columns[col_spc]
        nameevodf.at[0, col_name] = withnameoriginaldf.iloc[row1, 0]
        nameevodf.at[1, col_name] = withnameoriginaldf.iloc[row2, 0]

nameevodf.to_excel('final_result.xlsx')


{0: [0, 11], 1: [4, 16], 2: [3, 13], 3: [1, 19], 4: [7, 16], 5: [3, 6], 6: [8, 12], 7: [1, 14], 8: [7, 19], 9: [1, 19], 10: [5, 16], 11: [0, 18], 12: [13, 16], 13: [1, 3], 14: [6, 12], 15: [3, 12], 16: [1, 18], 17: [6, 12], 18: [1, 5], 19: [0, 14], 20: [16, 17], 21: [5, 8], 22: [0, 7], 23: [5, 16], 24: [1, 8], 25: [0, 4], 26: [2, 12], 27: [7, 11], 28: [11, 15], 29: [5, 15], 30: [13, 14], 31: [2, 18], 32: [2, 18], 33: [15, 16], 34: [5, 11], 35: [13, 17], 36: [2, 12], 37: [7, 15], 38: [5, 11], 39: [5, 14], 40: [3, 17], 41: [9, 10], 42: [12, 18], 43: [4, 15], 44: [13, 17], 45: [2, 18], 46: [0, 7], 47: [7, 18], 48: [17, 18], 49: [6, 14], 50: [0, 11], 51: [2, 3], 52: [3, 6], 53: [2, 6], 54: [11, 14], 55: [6, 11], 56: [0, 16], 57: [1, 7], 58: [2, 3], 59: [6, 12]}
2.0


(59, [6, 12])

In [16]:


print(ndf)

    Adil Harroud   Amanda Black AmanPreet Badhwar  Angela Auriat  \
0   Considerable     Moderate**      Considerable   Considerable   
1  Considerable*  Considerable*     Considerable*  Considerable*   

  Anna MacKinnon Aurelie de Rus Jacquet Britt Drogemoller      Chao Zheng  \
0       Moderate         Considerable**          Moderate  Considerable**   
1   Considerable          Considerable*     Considerable*        Moderate   

  Chelsea Ekstrand    Cindy Barha  ...   Ryan Hoiland    Sam Workenhe  \
0        Moderate*   Considerable  ...       Moderate   Considerable*   
1     Considerable  Considerable*  ...  Considerable*  Considerable**   

     Scott Yuzwa Shraddha Pai   Silvia Pozzi Stephen Glasgow  \
0   Considerable   Moderate**  Considerable*        Moderate   
1  Considerable*    Moderate*     Moderate**        Moderate   

  Vanessa Gonçalves Vincy  Chan      Yang Zhou Yoshiaki Tanaka  
0        Moderate**   Moderate*   Considerable    Considerable  
1      Considerable 

In [22]:
ndf.to_excel('output_type.xlsx', index=False)

In [38]:
# For each column index and its corresponding [start, end] indices, slice the string values.


  Adil Harroud Amanda Black AmanPreet Badhwar Angela Auriat Anna MacKinnon  \
0  Considerabl          NaN               NaN           NaN            NaN   
1     Moderate          NaN             erate   onsiderable              e   
2          NaN          NaN        flict of I           NaN            NaN   
3          NaN          NaN         siderable       oderate            NaN   
4          NaN       rate**               NaN           NaN            NaN   

  Aurelie de Rus Jacquet Britt Drogemoller     Chao Zheng Chelsea Ekstrand  \
0                    NaN               NaN            NaN              NaN   
1                    NaN               NaN  onsiderable**              e**   
2                    fli                          oderate              NaN   
3                    sid               of         oderate              NaN   
4                    NaN               NaN            NaN              NaN   

          Cindy Barha  ... Ryan Hoiland Sam Workenhe Scott Yuz

In [18]:
print(withnameoriginaldf)

                          Reviewers     Adil Harroud   Amanda Black  \
0               Karl Klein, MD (AB)     Considerable            NaN   
1               Lindsay Cahill (NL)         Moderate            NaN   
2       Janelle Drouin-Ouellet (QC)              NaN            NaN   
3                 Jill Stobart (MB)              NaN            NaN   
4         Jennifer Graves, MD (USA)              NaN     Moderate**   
5               Andrew Tasker (PEI)              NaN            NaN   
6                Annie Ciernia (BC)              NaN            NaN   
7             Isabelle Boileau (ON)              NaN            NaN   
8                  Simon Beggs (UK)              NaN            NaN   
9             Anne Warlaumont (USA)              NaN            NaN   
10               Rishi Ganesan (ON)              NaN            NaN   
11  Carlos Camara-Lemarroy, MD (AB)    Considerable*            NaN   
12              Federico Gaiti (ON)       Moderate**            NaN   
13    

                      Adil Harroud               Amanda Black  \
0              Karl Klein, MD (AB)  Jennifer Graves, MD (USA)   
1  Carlos Camara-Lemarroy, MD (AB)        Brian Christie (BC)   

      AmanPreet Badhwar          Angela Auriat         Anna MacKinnon  \
0     Jill Stobart (MB)    Lindsay Cahill (NL)  Isabelle Boileau (ON)   
1  Caroline Menard (QC)  Gillian Einstein (ON)    Brian Christie (BC)   

  Aurelie de Rus Jacquet    Britt Drogemoller           Chao Zheng  \
0      Jill Stobart (MB)     Simon Beggs (UK)  Lindsay Cahill (NL)   
1     Annie Ciernia (BC)  Federico Gaiti (ON)    Derya Sargin (AB)   

        Chelsea Ekstrand            Cindy Barha  ...  \
0  Isabelle Boileau (ON)    Lindsay Cahill (NL)  ...   
1  Gillian Einstein (ON)  Gillian Einstein (ON)  ...   

                      Ryan Hoiland                 Sam Workenhe  \
0              Karl Klein, MD (AB)  Janelle Drouin-Ouellet (QC)   
1  Carlos Camara-Lemarroy, MD (AB)            Jill Stobart (MB)   

  

     Adil Harroud   Amanda Black     AmanPreet Badhwar  Angela Auriat  \
0    Considerable            NaN                   NaN            NaN   
1        Moderate            NaN              Moderate   Considerable   
2             NaN            NaN  Conflict of Interest            NaN   
3             NaN            NaN          Considerable       Moderate   
4             NaN     Moderate**                   NaN            NaN   
5             NaN            NaN              Moderate       Moderate   
6             NaN            NaN                   NaN            NaN   
7             NaN            NaN                   NaN       Moderate   
8             NaN            NaN                   NaN            NaN   
9             NaN            NaN                   NaN            NaN   
10            NaN            NaN                   NaN            NaN   
11  Considerable*            NaN              Moderate            NaN   
12     Moderate**            NaN              Moder

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.

In [18]:
np.set_printoptions(threshold=np.inf)

In [27]:
simmatrix

array([[   5., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100., -100.,    1., -100., -100., -100., -100., -100., -100.,
        -100.,    1., -100., -100.,    1., -100., -100.,    5., -100.,
        -100., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100., -100., -100., -100., -100., -100., -100., -100., -100.,
        -100.,    1., -100., -100., -100.,    1., -100., -100., -100.,
        -100., -100.,    1., -100., -100., -100.],
       [   1., -100.,    1.,    5.,    1., -100., -100.,    5.,    1.,
           5., -100.,    1.,    1.,    5., -100., -100.,    5., -100.,
           1.,    1.,    1.,    1., -100., -100.,    5., -100., -100.,
           1.,    1., -100., -100., -100., -100., -100., -100.,    1.,
        -100.,    1., -100.,    1., -100., -100.,    1., -100., -100.,
        -100.,    1., -100., -100., -100., -100., -100., -100., -100.,
        -100.,    1.,    1.,    1., -100., -100.],
       [-100., -100., -100., -100., -100., -10

AttributeError: 'auto_assigner' object has no attribute 'fair_assignment'